In [1]:
import os
import pandas as pd

csv_path = r"C:\Users\SANJANA\Downloads\Dataset (1)\Dataset\Recordings\audio__details.csv"

audio_folder_path = r"C:\Users\SANJANA\Downloads\Dataset (1)\Dataset\Recordings\Train"


df = pd.read_csv(csv_path)
    

df['File_name'] = df['File_name'].apply(lambda x: os.path.abspath(os.path.join(audio_folder_path, os.path.basename(x))))


df.to_csv(csv_path, index=False)

In [2]:
from datasets import load_dataset, Audio


dataset = load_dataset('csv', data_files= r"C:\Users\SANJANA\Downloads\Dataset (1)\Dataset\Recordings\audio__details.csv")

dataset = dataset.cast_column('File_name', Audio(sampling_rate=16000))

dataset = dataset.rename_column('File_name', 'audio')
dataset = dataset.rename_column('phrase', 'sentence')

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base")

def prepare_dataset(batch):
    # Process audio to input features
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    # Process transcription to labels
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

# Specify only the columns you need to keep
columns_to_remove = [
    'audio_clipping', 'audio_clipping:confidence', 'background_noise_audible', 
    'background_noise_audible:confidence', 'overall_quality_of_the_audio', 
    'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download', 
    'prompt', 'writer_id'
]

# Filter out columns that are not in the dataset
columns_to_remove = [col for col in columns_to_remove if col in dataset.column_names]

# Apply the processing function to the dataset
dataset = dataset.map(prepare_dataset, remove_columns=columns_to_remove)


Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [4]:
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.nn.utils.rnn import pad_sequence


model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")


training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=False,
    save_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
)

C:\Users\SANJANA\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
class DataCollatorForWhisper:
    def __call__(self, features):
        
        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        labels = [torch.tensor(feature["labels"]) for feature in features]

        
        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_features": input_features_padded,
            "labels": labels_padded
        }

In [6]:
data_collator = DataCollatorForWhisper()

In [7]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=processor.tokenizer,  
    data_collator=data_collator,
)

In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\SANJANA\_netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


C:\Users\SANJANA\anaconda3\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=9, training_loss=4.4827291700575085, metrics={'train_runtime': 639.4676, 'train_samples_per_second': 0.507, 'train_steps_per_second': 0.014, 'total_flos': 1.81607989248e+16, 'train_loss': 4.4827291700575085, 'epoch': 2.571428571428571})

In [9]:
model.save_pretrained("whisper-finetuned_with_voice_dataset")
processor.save_pretrained("whisper-finetuned_with_voice_dataset")

[]